In [0]:
import numpy as np
np.__version__

Out[1]: '1.21.5'

In [0]:
pip install tensorflow

Python interpreter will be restarted.
  Using cached tensorflow-2.13.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (524.1 MB)
  Using cached termcolor-2.3.0-py3-none-any.whl (6.9 kB)
  Using cached flatbuffers-23.5.26-py2.py3-none-any.whl (26 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached grpcio-1.56.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.2 MB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached protobuf-4.23.4-cp37-abi3-manylinux2014_x86_64.whl (304 kB)
  Using cached tensorflow_estimator-2.13.0-py2.py3-none-any.whl (440 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached tensorboard-2.13.0-py3-none-any.whl (5.6 MB)
  Using cached tensorflow_io_gcs_filesystem-0.32.0-cp39-cp39-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (2.4 MB)
  Using cached libclang-16.0.0-py2.py3-none-manylinux2010_x86_64.whl (22.9 MB)
  Using cached absl_py-1.4.0-py3-none-any.whl (126 kB)
  Using c

In [0]:
import tensorflow as tf

/databricks/python/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [0]:
! pip install transformers
! pip install dlt

  Using cached transformers-4.30.2-py3-none-any.whl (7.2 MB)
  Using cached huggingface_hub-0.16.4-py3-none-any.whl (268 kB)
  Using cached PyYAML-6.0-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (661 kB)
  Using cached safetensors-0.3.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
  Using cached regex-2023.6.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (769 kB)
  Using cached tokenizers-0.13.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)
  Using cached fsspec-2023.6.0-py3-none-any.whl (163 kB)
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-1efa2fcd-8a32-4af4-afa3-ecac78e42415/bin/python -m pip install --upgrade pip' command.
  Using cached dlt-0.3.3-py3-none-any.whl (373 kB)
  Using cached orjson-3.9.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (138 kB)
  Using cached pendulum-2.1

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, monotonically_increasing_id
from pyspark.sql.types import StringType
from transformers import TFBertForSequenceClassification, BertTokenizer, pipeline

# Load the fine-tuned ALBERT model
model_lib = "vineet1409/fine-tuned-AlBERT"  
albert_classifier = pipeline("text-classification", model=model_lib, tokenizer=model_lib)

# Load the bio-clinical-BERT model and tokenizer
clinicalbert_model = TFBertForSequenceClassification.from_pretrained("vineet1409/fine-tuned-bioclinical-BERT")
clinicalbert_tokenizer = BertTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

Some layers from the model checkpoint at vineet1409/fine-tuned-AlBERT were not used when initializing TFAlbertForSequenceClassification: ['dropout_9']
- This IS expected if you are initializing TFAlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFAlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFAlbertForSequenceClassification were initialized from the model checkpoint at vineet1409/fine-tuned-AlBERT.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFAlbertForSequenceClassification for predictions without further training.
Some layers from the model checkpoint at vin

In [0]:
# Silver Layer- ALBERT

import dlt

def run_analysis(text):
    try:
        predictions = albert_classifier(text)  # Assuming albert_classifier is defined elsewhere
        predicted_label = predictions[0]['label']
        if predicted_label == 'LABEL_1':
            result = 'Suicidal'
        elif predicted_label == 'LABEL_0':
            result = 'Non-suicidal'
        return result
    except Exception:
        result = ' '
        return result
    
try:
    @dlt.table(
    comment="Silver_Layer_albert: Custom LLM {ALBERT} Predictions on dataset."
    )
    def mental_health_dlt_albert_silver():  
        df = dlt.read('mental_health_dlt_bronze')
        pandas_df = df.toPandas()
        pandas_df = pandas_df.iloc[:20,] # taking only 20 records as of now
        texts = pandas_df['text'].tolist()

        results = []
        for text in texts:
            result = run_analysis(text)
            results.append(result)

        pandas_df['albert-predictions'] = results
        spark_df_silver_albert = spark.createDataFrame(pandas_df)

        return spark_df_silver_albert

except AttributeError:
    class dlt:  
        def table(comment, **options): # Mock the @dlt.table attribute so that it is seen as syntactically valid below  
            def _(f):  
                pass  
            return _; 



In [0]:
## Silver Layer clinicalbert
import dlt 
  
try:
    @dlt.table(
    comment="Silver_Layer_clinicalbert: Custom LLM {clinicalbert} Predictions on dataset."
    )
    def mental_health_dlt_clinicalbert_silver():  
        df = dlt.read('mental_health_dlt_bronze')
        pandas_df_cb = df.toPandas()
        pandas_df_cb = pandas_df_cb.iloc[:20,]
        texts = pandas_df_cb['text'].tolist()

        results = []
        for text in texts:
            encoded_input = clinicalbert_tokenizer.batch_encode_plus(
                            [text],
                            padding=True,
                            truncation=True,
                            return_tensors="tf",
                            max_length=128
                            ) 
            predict = clinicalbert_model.predict(encoded_input['input_ids'])
            predicted_labels = np.argmax(predict.logits, axis=1)
            class_names = ["Non-suicidal", "Suicidal"]
            predicted_classes = [class_names[label] for label in predicted_labels] 
            output = predicted_classes[0]
            results.append(output)

        pandas_df_cb['clinicalbert-predictions'] = results
        spark_df_silver_clinicalbert = spark.createDataFrame(pandas_df_cb)
        
        return spark_df_silver_clinicalbert


except AttributeError:
    class dlt:  
        def table(comment, **options): # Mock the @dlt.table attribute so that it is seen as syntactically valid below  
            def _(f):  
                pass  
            return _; 

